In [208]:
# General
import numpy as np
import matplotlib.pyplot as plt

# PyTorch
import torch.cuda
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# classic ML
from sklearn.metrics import accuracy_score
import pandas as pd

## Константы 

In [209]:
img_size = 256
batch_size = 7
num_epochs = 5
learning_rate = 0.001

##  Проверка возможности обучения на GPU

In [210]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Загрузка изображенией с применением трансформаций

Сначала просто открываем картинки без каких-либо трансформаций

In [211]:
trfrms = [
    transforms.Resize(size=(img_size, img_size)),
    transforms.ToTensor()
]

In [212]:
train_folder = ImageFolder(root='train_115', transform=transforms.Compose(trfrms))
test_folder = ImageFolder(root='test_115', transform=transforms.Compose(trfrms))

Сохраняем количество классов и словарь названий

In [213]:
num_classes = len(train_folder.classes)

classes_map = dict((v,k) for k,v in train_folder.class_to_idx.items())

Создаем список трансформаций, которые будут применяться после открытия, но до конвертации изображения в тензор

In [214]:
transformation_list = [
    transforms.RandomRotation(degrees=45),
    transforms.RandomHorizontalFlip(1),
    transforms.RandomVerticalFlip(1)
#     transforms.RandomErasing()
]

В цикле открываем папку с указанными трансформациями и прибавляем к уже существующим картинкам

In [215]:
for transformation in transformation_list:
    transes = []
    transes.append(transforms.Resize(size=(img_size, img_size)))
    transes.append(transformation)
    transes.append(transforms.ToTensor())
    
    tr_folder = ImageFolder(root='train_115', transform=transforms.Compose(transes))
    
    train_folder += tr_folder
    
    print(f"{transformation} done. Len(train_folder) = {len(train_folder)}.")

RandomRotation(degrees=(-45, 45), resample=False, expand=False) done. Len(train_folder) = 1312.
RandomHorizontalFlip(p=1) done. Len(train_folder) = 1968.
RandomVerticalFlip(p=1) done. Len(train_folder) = 2624.


---

In [216]:
train_loader = DataLoader(train_folder, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_folder, batch_size=batch_size, shuffle=True)

In [217]:
stat_out = round(len(train_loader) / 5)
stat_out

75

## Определение архитектуры

In [218]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 61 * 61, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 8)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 61 * 61)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [219]:
net = Net()
net.to(device)
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=59536, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=8, bias=True)
)


In [220]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

## Обучение 

In [221]:
%%time
for epoch in range(num_epochs):  # loop over the dataset multiple times
    
    print(f"Epoch {epoch + 1} ------------")
    running_loss = 0.0
    
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % stat_out == (stat_out - 1):
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / stat_out}")
            running_loss = 0.0

print('Finished Training')

Epoch 1 ------------
[1, 75] loss: 2.08569104830424
[1, 150] loss: 2.083060636520386
[1, 225] loss: 2.079458656311035
[1, 300] loss: 2.0794859155019125
[1, 375] loss: 2.079704844156901
Epoch 2 ------------
[2, 75] loss: 2.0761066977183025
[2, 150] loss: 2.0739735762278237
[2, 225] loss: 2.0729218403498333
[2, 300] loss: 2.0772350788116456
[2, 375] loss: 2.072647574742635
Epoch 3 ------------
[3, 75] loss: 2.0677400096257528
[3, 150] loss: 2.0605784686406454
[3, 225] loss: 2.0386090151468914
[3, 300] loss: 1.993476282755534
[3, 375] loss: 2.0083731031417846
Epoch 4 ------------
[4, 75] loss: 1.9585496155420938
[4, 150] loss: 1.9847379191716512
[4, 225] loss: 1.9215258979797363
[4, 300] loss: 1.9564159059524535
[4, 375] loss: 1.9300690873463948
Epoch 5 ------------
[5, 75] loss: 1.9150438117980957
[5, 150] loss: 1.9052933279673259
[5, 225] loss: 1.922324914932251
[5, 300] loss: 1.923303836186727
[5, 375] loss: 1.8941141653060913
Finished Training
Wall time: 3min 2s


Сохранение выученных коэффициентов для возможности дальнейшей загрузки

In [222]:
coeff_path = f"nn_coeffs/fixed_count_and_transformations/own_cnn_{img_size}x{img_size}.pth"
torch.save(net.state_dict(), coeff_path)

## Оценка accuracy

In [223]:
%%time

result_dict = {
    "real" : [],
    "predicted" : []
}

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        for i in range(len(labels)):
            result_dict["real"].append(classes_map[labels[i].item()])
            result_dict["predicted"].append(classes_map[predicted[i].item()])

result_df = pd.DataFrame(result_dict)
result_df["answer"] = result_df.apply(lambda x: int(x["real"] == x["predicted"]), axis=1)

total_acc = accuracy_score(result_df["real"], result_df["predicted"]) * 100
print(f"Total accuracy: {round(total_acc, 2)}%")
print("---------------------")

for img_class in result_df["real"].unique():
    class_df = result_df[result_df["real"] == img_class]
    class_acc = accuracy_score(class_df["real"], class_df["predicted"]) * 100
    print(f"{img_class} ({len(class_df)}) : {round(class_acc, 2)}%")

Total accuracy: 23.11%
---------------------
melanoma (33) : 9.09%
vascular lesion (33) : 3.03%
dermatofibroma (33) : 0.0%
squamous cell carcinoma (33) : 6.06%
basal cell carcinoma (33) : 57.58%
pigmented benign keratosis (33) : 0.0%
nevus (33) : 63.64%
actinic keratosis (33) : 45.45%
Wall time: 3.41 s
